In [78]:
!pip install -q sklearn
!pip install pyarrow

In [79]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [198]:
dataframe = pd.read_feather('./Datasets/local/cicdos2017')

In [199]:
dataframe.head()

,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,bwd packet length max,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,11252489,26,61,1294,27134,240,0,49.769230,55.501572,1460,...,0,42720.0,0.0,42720,42720,5018174.0,0.0,5018174,5018174,0
1,5227222,14,39,776,3096,88,0,55.428570,25.193754,720,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
2,9249484,17,45,822,5422,240,0,48.352940,65.100830,784,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
3,2515425,1,1,0,0,0,0,0.000000,0.000000,0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
4,3826588,3,5,596,167,596,0,198.666672,344.100769,167,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0


In [200]:
dataframe['target'] = dataframe['label']
dataframe = dataframe.drop(columns=['label'])

In [201]:
dataframe.columns = dataframe.columns.str.replace(' ', '_')
len(dataframe.columns)

77

In [202]:
train, test = train_test_split(dataframe, test_size=0.33)
train, val = train_test_split(train, test_size=0.33)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

130871 train examples
64460 validation examples
96209 test examples


In [1]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [209]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of min packet length:', feature_batch['min_packet_length'])
    print('A batch of targets:', label_batch )

Every feature: ['flow_duration', 'total_fwd_packets', 'total_backward_packets', 'total_length_of_fwd_packets', 'total_length_of_bwd_packets', 'fwd_packet_length_max', 'fwd_packet_length_min', 'fwd_packet_length_mean', 'fwd_packet_length_std', 'bwd_packet_length_max', 'bwd_packet_length_min', 'bwd_packet_length_mean', 'bwd_packet_length_std', 'flow_bytes/s', 'flow_packets/s', 'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_total', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_total', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'fwd_header_length', 'bwd_header_length', 'fwd_packets/s', 'bwd_packets/s', 'min_packet_length', 'max_packet_length', 'packet_length_mean', 'packet_length_std', 'packet_length_variance', 'fin_flag_count', 'syn_flag_count', 'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'urg_flag_count', 'cwe_flag_count', 'ece_flag_count', 'd

In [210]:
feature_columns = []

# numeric cols
for header in dataframe.columns:
    if header!='target':
        feature_columns.append(feature_column.numeric_column(header))

In [211]:
batch_size = 500
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

77 --> 6/10 * 77 +/-= 46

In [213]:
from tensorflow.keras.layers import Lambda
from tensorflow import expand_dims

model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Reshape((-1, 76), input_shape=(76,)),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(0.5),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(0.5),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(0.5),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(0.5),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(0.1),
    layers.LSTM(46, return_sequences = True),
    #layers.Dropout(.1),
    layers.Dense(1)
])

opt = tf.keras.optimizers.RMSprop(learning_rate=0.01, rho=0.05, momentum=0.0, epsilon=1e-08, centered=False,name='RMSprop')
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=200, batch_size=500)

Epoch 1/200
262/262 [==============================] - 16s 60ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 2/200
262/262 [==============================] - 13s 51ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 3/200
262/262 [==============================] - 13s 48ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 4/200
262/262 [==============================] - 12s 46ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 5/200
262/262 [==============================] - 12s 45ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 6/200
262/262 [==============================] - 12s 44ms/step - loss: 1.7521e-08 - accuracy: 0.8530 - val_loss: 1.7401e-08 - val_accuracy: 0.8540
Epoch 7/200
262/262 [==============================] - 12s 45ms/step - loss: 1.752